In [8]:
import os
import sys
import urllib.request
import pandas as pd
import datetime
import pymysql
import re
import json

In [9]:
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"

url = "https://openapi.naver.com/v1/datalab/search";
body = "{\"startDate\":\"2016-01-01\",\"endDate\":\"2020-09-30\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\"경제\",\"keywords\":[\"경제\"]}]}";

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    scrapped = response_body.decode('utf-8')
else:
    print("Error Code:" + rescode)

In [10]:
jsonResult = json.loads(scrapped)
print(jsonResult['results'][0]['data'])

od': '2019-07-13', 'ratio': 14.49421}, {'period': '2019-07-14', 'ratio': 17.96678}, {'period': '2019-07-15', 'ratio': 25.88491}, {'period': '2019-07-16', 'ratio': 22.14393}, {'period': '2019-07-17', 'ratio': 23.16725}, {'period': '2019-07-18', 'ratio': 21.50645}, {'period': '2019-07-19', 'ratio': 18.40295}, {'period': '2019-07-20', 'ratio': 12.69921}, {'period': '2019-07-21', 'ratio': 14.36}, {'period': '2019-07-22', 'ratio': 18.83912}, {'period': '2019-07-23', 'ratio': 18.83912}, {'period': '2019-07-24', 'ratio': 18.08421}, {'period': '2019-07-25', 'ratio': 19.82888}, {'period': '2019-07-26', 'ratio': 17.12799}, {'period': '2019-07-27', 'ratio': 13.55477}, {'period': '2019-07-28', 'ratio': 13.35346}, {'period': '2019-07-29', 'ratio': 30.73309}, {'period': '2019-07-30', 'ratio': 18.26874}, {'period': '2019-07-31', 'ratio': 18.26874}, {'period': '2019-08-01', 'ratio': 18.30229}, {'period': '2019-08-02', 'ratio': 34.74249}, {'period': '2019-08-03', 'ratio': 19.81211}, {'period': '2019-08

In [11]:
def extract_year_and_month(period):
    year = period.split('-')[0]
    month = period.split('-')[1]
    day = period.split('-')[2]
    return year, month, day

# DB 연동 및 table 생성

In [20]:
conn = pymysql.connect(host = "127.0.0.1", user = [USER], passwd = [PASSWORD], db = [DATABASE])
cur = conn.cursor()

In [21]:
"""
검색어 트랜드 정보 저장할 table 생성
"""

create_table_query = """
CREATE TABLE IF NOT EXISTS naver_counting(
    id BIGINT(7) NOT NULL AUTO_INCREMENT,
    year bigint(4) NOT NULL,
    month bigint(2) NOT NULL,
    day bigint(2) NOT NULL,
    cnt double,
    primary key(id) )
    charset=utf8mb4;
"""
cur.execute(create_table_query)

0

In [22]:
cur.execute("desc naver_counting")
table = cur.fetchall()
pd.DataFrame(list(table))

,0,1,2,3,4,5
0,id,bigint(7),NO,PRI,None,auto_increment
1,year,bigint(4),NO,,None,
2,month,bigint(2),NO,,None,
3,day,bigint(2),NO,,None,
4,cnt,double,YES,,None,


In [23]:
"""
db table에 저장
"""

for result in jsonResult['results'][0]['data']:
    query = """ 
    Insert into naver_counting (year, month, day, cnt) values (%d, %d, %d, %f) ;
    """
    year, month, day = extract_year_and_month(result["period"])
    naver_cnt = result["ratio"]

    mystring = (query % (int(year), int(month), int(day), float(naver_cnt)))
    print(mystring)
    cur.execute(mystring)

Insert into naver_counting (year, month, day, cnt) values (2020, 3, 6, 22.227810) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2020, 3, 7, 18.436500) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2020, 3, 8, 18.956550) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2020, 3, 9, 34.474080) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2020, 3, 10, 32.460990) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2020, 3, 11, 32.662300) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2020, 3, 12, 44.690480) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2020, 3, 13, 73.074980) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2020, 3, 14, 29.105850) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2020, 3, 15, 26.488840) ;
    
 
    Insert into naver_counting (year, month, day

In [24]:
conn.commit()
cur.close()
conn.close()